<a href="https://colab.research.google.com/github/Hoangthang017/CS336.L11/blob/master/Crawler/crawler_article_daa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**import thư viện**

In [35]:
# import thư viện 
from bs4 import BeautifulSoup
import urllib.request

##**Home page**


In [36]:
url_hp = 'https://daa.uit.edu.vn'

##**Functine kiểm tra thông báo trong 1 tháng**

In [37]:
import datetime
import pytz
def check_time(date_str):

  # lấy ngày giờ hiện tại
  ## Dạng native date
  date_time_now = datetime.datetime.now()

  ## Dạng utc
  UTC = pytz.utc
  date_time_utc_now = UTC.localize(date_time_now)

  ## Dạng 'Asia/Ho_Chi_Minh'
  VN_TZ = pytz.timezone('Asia/Ho_Chi_Minh')
  date_time_now = date_time_utc_now.astimezone(VN_TZ)

  ## chuyển thời gian hiện tại qua dạng chuỗi
  date_time_now = date_time_now.strftime("%d/%m/%Y %H:%M")

  # convert ngày cần kiểm tra từ string sang datetime
  format = '%d/%m/%Y %H:%M'
  date_time_now = datetime.datetime.strptime(date_time_now, format)
  date_time = datetime.datetime.strptime(date_str, format)

  # convert datetime sang timestamp
  time_stamp = date_time_now.timestamp() - date_time.timestamp() 

  # convert timestamp sang datetime
  timestamp_to_datetime = datetime.datetime.fromtimestamp(time_stamp)

  # lấy ngày tháng 
  date = timestamp_to_datetime.strftime("%d")
  month = timestamp_to_datetime.strftime("%m")
  ##year = timestamp_to_datetime.strftime("%y")

  # nếu vượt quá nữa năm không cần cào nữa
  if (int(month) > 6):
    flag = False
  else:
    flag = True

  # kiểm tra có vượt 30 ngày hay không ?
  if (int(date) == 31 or int(month) >= 2):
    return [False,flag]
  return [True,flag]

##**Function craw data từ trang con**

In [38]:
def crawl_daa_subpage(url_sp):
  print("Loading...\n")

  # Các thông báo được craw
  notice = []

  # biến kiểm soát
  flag = True
  i = 0

  # vòng lặp
  while(flag):

    # get and request url subpage
    url = url_sp+ "?page=" + str(i)
    page = urllib.request.urlopen(url)
    content_more_link = BeautifulSoup(page, 'html.parser')

    # tìm tất cả thẻ thẻ div của class content chứa nội dung thông báo
    content = content_more_link.find_all('div',class_ = "content")

    # tìm cái thông báo phù hợp trong các class content
    for ct in content:

      # tìm tất cả bài báo có trong content
      article = ct.find_all('article')

      # vòng lặp lấy thông tin của article
      for art in article:

        # lấy thời gian bài viết
        day= art.find('div',class_ = "submitted").text.split()
        date_time = day[1] + " "+ day[3]

        # kiểm tra bài viết có nằm trong 1 tháng gần nhất hay không ?
        check = check_time(date_time)
        if (check[0]):
          # lấy title bài viết
          title = art.find('span').get('content')

          # lấy nội dung
          content_art =  art.find('div',class_ = 'content').text.strip()

          # lấy link bài viết
          link = art.find('ul',class_ = "links inline").find('a').get('href')
          link = url_hp + link

          # push article phù hợp vào 
          notice.append([title,date_time,content_art,link])

        else:

          # kiểm tra xem bài viết vượt quá nữa năm hay chưa 
          ## nếu vượt quá dừng craw data
          if (check[1] == False):
            flag = False

    # tăng số page của trang
    i += 1

  print("Done!!!")
  return notice

##**Vì các thông báo đều được tổng hợp ở page thông báo nên chỉ cần crawl ở page này**

In [39]:
url_sp = "https://daa.uit.edu.vn/thongbao"
Result = crawl_daa_subpage(url_sp)

Loading...

Done!!!


##**Đưa dữ liệu vào file csv**

In [40]:
import csv

with open('/content/Notice_Daa.csv', mode='w') as file:
    writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(["Title","Time","Content","Link"])
    writer.writerows(Result)